In [39]:
gpu_info = !nvidia-smi
gpu_info = gpu_info[:10]
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Mon Mar  8 19:03:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.102.04   Driver Version: 450.102.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  TITAN V             On   | 00000000:17:00.0 Off |                  N/A |
| 29%   42C    P8    26W / 250W |   5069MiB / 12066MiB |      0%      Default |


In [40]:
# from google.colab import drive
# drive.mount('/content/drive')

In [41]:
from varname import nameof

import sys
import torch
import os
import numpy as np
import torch.nn as nn
import torch.optim as optim

from tqdm.notebook import tqdm_notebook as tq

from torch.autograd import Variable
from torchvision.transforms import transforms
from torchvision.utils import save_image, make_grid
from torch.optim.lr_scheduler import StepLR, ExponentialLR
from pytorch_model_summary import summary
from tensorboardX import SummaryWriter

In [42]:
from src.dataset.utils import SavePath
from src.dataset.dataset import Cifar
from src.config import TrainConfig
from src.pytorch_msssim import MSSSIM, ssim

In [43]:
base_path = !pwd
base_path = base_path[0] + '/'

In [44]:
args = TrainConfig( base_path,              # project directory path
                    n_epochs = 200,         # number of epochs to train (default: 100)
                    batch_size = 128,       # input batch size for training (default: 128)
                    lr = 1e-3,            # learning rate (default: 0.0001)
                    dim_h = 128,            # hidden dimension (default: 128)')
                    n_z = 128,                # hidden dimension of z (default: 8)
                    LAMBDA = 10,            # regularization coef term (default: 10)
                    sigma = 1,              # variance of hidden dimension (default: 1)
                    n_channel = 3,          # input channels (default: 1)
                    img_size = 32 )         # image size

In [45]:
def unfreeze_params(module: nn.Module):
    for p in module.parameters():
        p.requires_grad = True

def freeze_params(module: nn.Module):
    for p in module.parameters():
        p.requires_grad = False

In [46]:
def save_models(model_path, epoch_no, models):
    print("Saving models")
    for model_name, model in models.items():
        torch.save(model.state_dict(), model_path + model_name + "_" + "%d.pth" % epoch_no)

def save_values_to_tensorboard(writer, epoch_no, values_dict: dict):
    for name, val in values_dict.items():
        if type(val) == dict:
            writer.add_scalars(name, val, epoch_no)
        else:
            writer.add_scalar(name, val, epoch_no)

def save_images_to_tensorboard(writer, epoch_no, image, imname='im'):
    writer.add_image(imname +'_{}'.format(epoch_no), image, epoch_no)

In [47]:
sp = SavePath(args)

/home/pr/synth.data/Autoencoder/outs/Mon-Mar--8-19-03-07-2021/


In [48]:
transform = None # dont normalize

cdl = Cifar(args)
train_loader = cdl.get_data_loader(True, transform, [0,1,2,3,4])
test_loader = cdl.get_data_loader(False, transform, [0,1,2,3,4])

Files already downloaded and verified
Files already downloaded and verified


In [49]:
from src.models.model_cifar import Encoder as ConvEncoder
from src.models.model_cifar import Decoder as ConvDecoder
from src.models.model_cifar import GanDiscriminator

In [50]:
class ZDiscriminator(nn.Module):
    def __init__(self, args):
        super(ZDiscriminator, self).__init__()

        self.n_channel = args.n_channel
        self.dim_h = args.dim_h
        self.n_z = args.n_z

        self.main = nn.Sequential(
            nn.Linear(self.n_z, self.dim_h * 4),
            nn.ReLU(True),
            nn.Linear(self.dim_h * 4, self.dim_h * 4),
            nn.ReLU(True),
            nn.Linear(self.dim_h * 4, self.dim_h * 2),
            nn.ReLU(True),
            nn.Linear(self.dim_h * 2, self.dim_h // 2),
            nn.ReLU(True),
            nn.Linear(self.dim_h // 2, self.dim_h // 4),
            nn.ReLU(True),
            nn.Linear(self.dim_h // 4, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.main(x)
        x = x.squeeze()
        return x

In [51]:
conv_encoder, conv_decoder = ConvEncoder(args).cuda(), ConvDecoder(args).cuda()
gan_discriminator = GanDiscriminator(args).cuda()
z_discriminator = ZDiscriminator(args).cuda()

mse_loss_fn = nn.MSELoss().cuda()
adversarial_loss_fn = nn.BCELoss().cuda()

opt_encoder = optim.Adam(conv_encoder.parameters(), lr = args.lr)
opt_decoder = optim.Adam(conv_decoder.parameters(), lr = args.lr)
opt_discriminator = optim.Adam(gan_discriminator.parameters(), lr = args.lr)
opt_zdiscriminator = optim.Adam(z_discriminator.parameters(), lr = args.lr)
scheduler_opt_encoder = ExponentialLR(opt_encoder, gamma=0.1)
scheduler_opt_decoder = ExponentialLR(opt_decoder, gamma=0.1)
scheduler_opt_discriminator = ExponentialLR(opt_discriminator, gamma=0.1)
scheduler_opt_zdiscriminator = ExponentialLR(opt_zdiscriminator, gamma=0.1)

In [52]:
# print(summary(conv_encoder, torch.zeros((1, 3, 32, 32)).cuda(), show_input=True, show_hierarchical=False))

In [53]:
def load_models(checkpoint_path, checkpoint):
    lp = SavePath(args, checkpoint_path)
    _, _, model_load_path = lp.get_save_paths()
    conv_encoder.load_state_dict(torch.load(model_load_path + "/conv_encoder_{}.pth".format(checkpoint)))
    conv_decoder.load_state_dict(torch.load(model_load_path + "/conv_decoder_{}.pth".format(checkpoint)))
    gan_discriminator.load_state_dict(torch.load(model_load_path + "/gan_discriminator_{}.pth".format(checkpoint)))

In [54]:
checkpoint = 0
if checkpoint:
    load_models('/home/pr/synth.data/Autoencoder/outs/Mon-Mar--1-12-35-57-2021/', checkpoint)

reconstr_loss, disc_loss = [], []
norms = []
writer = SummaryWriter(log_dir = sp.results_path + "logs")

In [55]:
image_path, list_path, model_path = sp.get_save_paths()

In [56]:
# one = torch.tensor(1)
# mone = one * -1
# if torch.cuda.is_available():
#     one, mone = one.cuda(), mone.cuda()

In [57]:
for epoch in range(checkpoint, checkpoint+args.n_epochs):
    pbar = tq(enumerate(train_loader))
    for step, (images, _) in pbar:
        
        current_batch_size = images.size()[0]        
        images = images.cuda()

        conv_encoder.zero_grad()
        conv_encoder.zero_grad()
        z_discriminator.zero_grad()
        
        # Adversarial ground truths
        ones = Variable(torch.cuda.FloatTensor(current_batch_size).fill_(1.0), requires_grad=False) # real
        zeros = Variable(torch.cuda.FloatTensor(current_batch_size).fill_(0.0), requires_grad=False) # fake

        # ======== Train Discriminator ======== #

#         freeze_params(decoder)
#         freeze_params(encoder)
        unfreeze_params(z_discriminator)

        z_prior = torch.randn(current_batch_size, args.n_z).cuda()
        d_prior = z_discriminator(z_prior)
        z_real = conv_encoder(images)
        d_real = z_discriminator(z_real.detach())
        
        # save norms of encodings
        norms.append([torch.norm(z_real, dim=1).mean().item()])

        # negate for gradient ascent
#         d_loss = -(torch.log(d_fake).mean() + torch.log(1 - d_real).mean())
#         d_loss = d_prior.mean() + (1 - d_real).mean()
        d_loss = adversarial_loss_fn(d_prior, ones).mean() + adversarial_loss_fn(d_real, zeros).mean()

        disc_loss.append(d_loss.item())

        d_loss.backward()

        opt_zdiscriminator.step()


        # ======== Train Generator ======== #

#         unfreeze_params(decoder)
#         unfreeze_params(encoder)
        freeze_params(z_discriminator)

#         z_real = encoder(images)
        x_recon = conv_decoder(z_real)
        d_real = z_discriminator(conv_encoder(Variable(images.data)))

        # recon_loss = criterion(x_recon, images) - (args.LAMBDA * (torch.log(d_real)).mean())

        recon_loss = 10 * mse_loss_fn(x_recon, images).mean()
        reg_loss = 0.005 * adversarial_loss_fn(d_real, ones).mean()
#         d_loss = args.LAMBDA * (d_real).mean()

        reconstr_loss.append([recon_loss.data.item(), reg_loss.data.item()])

        recon_loss.backward()
        reg_loss.backward()

        opt_encoder.step()
        opt_decoder.step()
        
        pbar.set_description("Recon Loss: [%.3f], Reg loss: [%.3f], disc loss: [%.3f]" % (
            recon_loss.item(), reg_loss.item(), d_loss.item()))

    if (epoch + 1) % 1 == 0:
        reconstr_loss_mean = np.mean(reconstr_loss, axis=0)
        print("Epoch: [%d/%d], Recon Loss: [%.3f], Reg loss: [%.3f], disc_loss: [%.3f], " %
                (epoch + 1, args.n_epochs, 
                 reconstr_loss_mean[0], reconstr_loss_mean[1], np.mean(disc_loss)))
        disc_loss.clear()
        reconstr_loss.clear()

    if (epoch + 1) % 1 == 0:

        batch_size = 64
        test_iter = iter(test_loader)
        test_images, _ = next(test_iter)
        test_images = Variable(test_images)[:64]
        
        z_real = conv_encoder(test_images.cuda()).detach()
        test_recon = conv_decoder(z_real)
        
        test_mse_loss = mse_loss_fn(test_images, test_recon.cpu()).data.item()

        random_sample = conv_decoder(torch.randn_like(z_real)).cpu().view(batch_size, 3, 32, 32)

        test_image_recon = torch.cat((test_images.view(batch_size, 3, 32, 32), 
                                      test_recon.cpu().view(batch_size, 3, 32, 32).data), axis=3)
        
        val_dict = {
        "train_recon_loss": {"recon": reconstr_loss_mean[0],
                            "reg": reconstr_loss_mean[1]
                            },
        "mse_ssim": {"train_mse": reconstr_loss_mean[0], 
                     "test_mse": test_mse_loss,
                     "train_ssim": ssim(images.cpu(), x_recon.cpu()).data.item()
                    },
        "norms": {"real": np.mean(norms),
                  "gaussian": torch.norm(z_prior, dim=1).mean().item()
                 }
        }
        
        norms.clear()
        
        save_values_to_tensorboard(writer, epoch + 1, val_dict)
        
        save_images_to_tensorboard(writer, epoch+1, make_grid(torch.cat((images, x_recon), dim=3)
                                                                           , normalize=False), 'train')
        save_images_to_tensorboard(writer, epoch+1, make_grid(test_image_recon, normalize=False), 'test')
        save_images_to_tensorboard(writer, epoch+1, make_grid(random_sample, normalize=False), 'sample')
        
        

    if (epoch + 1) % 25 == 0:
        models = {nameof(conv_encoder): conv_encoder, 
                  nameof(conv_decoder): conv_decoder}
        save_models(model_path, epoch+1, models)

0it [00:00, ?it/s]

Epoch: [1/200], Recon Loss: [0.650], Reg loss: [0.048], disc_loss: [0.309], 


0it [00:00, ?it/s]

Epoch: [2/200], Recon Loss: [0.623], Reg loss: [0.055], disc_loss: [0.067], 


0it [00:00, ?it/s]

...

Epoch: [198/200], Recon Loss: [0.254], Reg loss: [0.027], disc_loss: [0.138], 


0it [00:00, ?it/s]

Epoch: [199/200], Recon Loss: [0.275], Reg loss: [0.026], disc_loss: [0.144], 


0it [00:00, ?it/s]

Epoch: [200/200], Recon Loss: [0.302], Reg loss: [0.029], disc_loss: [0.109], 
Saving models


In [55]:
torch.norm(z_prior, dim=1).mean().item()

11.355522155761719

In [27]:
reg_loss

tensor(32.2346, device='cuda:0', grad_fn=<MulBackward0>)

In [ ]:
image = torch.cat((test_data[0].view(batch_size, 1, 28, 28), 
                                reconst.data), axis=3)

In [ ]:
image.shape

torch.Size([128, 1, 28, 56])

In [ ]:
np.mean(disc_loss, axis=0)

In [ ]:
disc_loss[0]

Output hidden; open in https://colab.research.google.com to view.

In [3]:
# encoder block (used in encoder and discriminator)
class EncoderBlock(nn.Module):
    def __init__(self, channel_in, channel_out):
        super(EncoderBlock, self).__init__()
        # convolution to halve the dimensions
        self.conv = nn.Conv2d(in_channels=channel_in, out_channels=channel_out, kernel_size=5, padding=2, stride=2,
                              bias=False)
        self.bn = nn.BatchNorm2d(num_features=channel_out, momentum=0.9)

    def forward(self, ten, out=False,t = False):
        # here we want to be able to take an intermediate output for reconstruction error
        if out:
            ten = self.conv(ten)
            ten_out = ten
            ten = self.bn(ten)
            ten = F.relu(ten, False)
            return ten, ten_out
        else:
            ten = self.conv(ten)
            ten = self.bn(ten)
            ten = F.relu(ten, True)
            return ten


# decoder block (used in the decoder)
class DecoderBlock(nn.Module):
    def __init__(self, channel_in, channel_out):
        super(DecoderBlock, self).__init__()
        # transpose convolution to double the dimensions
        self.conv = nn.ConvTranspose2d(channel_in, channel_out, kernel_size=5, padding=2, stride=2, output_padding=1,
                                       bias=False)
        self.bn = nn.BatchNorm2d(channel_out, momentum=0.9)

    def forward(self, ten):
        ten = self.conv(ten)
        ten = self.bn(ten)
        ten = F.relu(ten, True)
        return ten


class Encoder(nn.Module):
    def __init__(self, channel_in=3, z_size=128):
        super(Encoder, self).__init__()
        self.size = channel_in
        layers_list = []
        # the first time 3->64, for every other double the channel size
        for i in range(3):
            if i == 0:
                layers_list.append(EncoderBlock(channel_in=self.size, channel_out=64))
                self.size = 64
            else:
                layers_list.append(EncoderBlock(channel_in=self.size, channel_out=self.size * 2))
                self.size *= 2

        # final shape Bx256x8x8
        self.conv = nn.Sequential(*layers_list)
        self.fc = nn.Sequential(nn.Linear(in_features=8 * 8 * self.size, out_features=1024, bias=False),
                                nn.BatchNorm1d(num_features=1024,momentum=0.9),
                                nn.ReLU(True))
        # two linear to get the mu vector and the diagonal of the log_variance
        self.l_mu = nn.Linear(in_features=1024, out_features=z_size)
        self.l_var = nn.Linear(in_features=1024, out_features=z_size)

    def forward(self, ten):
        ten = self.conv(ten)
        ten = ten.view(len(ten), -1)
        ten = self.fc(ten)
        mu = self.l_mu(ten)
        logvar = self.l_var(ten)
        return mu, logvar

    def __call__(self, *args, **kwargs):
        return super(Encoder, self).__call__(*args, **kwargs)


class Decoder(nn.Module):
    def __init__(self, z_size, size):
        super(Decoder, self).__init__()
        # start from B*z_size
        self.fc = nn.Sequential(nn.Linear(in_features=z_size, out_features=8 * 8 * size, bias=False),
                                nn.BatchNorm1d(num_features=8 * 8 * size,momentum=0.9),
                                nn.ReLU(True))
        self.size = size
        layers_list = []
        layers_list.append(DecoderBlock(channel_in=self.size, channel_out=self.size))
        layers_list.append(DecoderBlock(channel_in=self.size, channel_out=self.size//2))
        self.size = self.size//2
        layers_list.append(DecoderBlock(channel_in=self.size, channel_out=self.size//4))
        self.size = self.size//4
        # final conv to get 3 channels and tanh layer
        layers_list.append(nn.Sequential(
            nn.Conv2d(in_channels=self.size, out_channels=3, kernel_size=5, stride=1, padding=2),
            nn.Tanh()
        ))

        self.conv = nn.Sequential(*layers_list)

    def forward(self, ten):

        ten = self.fc(ten)
        ten = ten.view(len(ten), -1, 8, 8)
        ten = self.conv(ten)
        return ten

    def __call__(self, *args, **kwargs):
        return super(Decoder, self).__call__(*args, **kwargs)


class Discriminator(nn.Module):
    def __init__(self, channel_in=3,recon_level=3):
        super(Discriminator, self).__init__()
        self.size = channel_in
        self.recon_levl = recon_level
        # module list because we need need to extract an intermediate output
        self.conv = nn.ModuleList()
        self.conv.append(nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True)))
        self.size = 32
        self.conv.append(EncoderBlock(channel_in=self.size, channel_out=128))
        self.size = 128
        self.conv.append(EncoderBlock(channel_in=self.size, channel_out=256))
        self.size = 256
        self.conv.append(EncoderBlock(channel_in=self.size, channel_out=256))
        # final fc to get the score (real or fake)
        self.fc = nn.Sequential(
            nn.Linear(in_features=8 * 8 * self.size, out_features=512, bias=False),
            nn.BatchNorm1d(num_features=512,momentum=0.9),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=512, out_features=1),
        )

    def forward(self, ten_orig, ten_predicted, ten_sampled, mode='REC'):
        if mode == "REC":
            ten = torch.cat((ten_orig, ten_predicted, ten_sampled), 0)
            for i, lay in enumerate(self.conv):
                # we take the 9th layer as one of the outputs
                if i == self.recon_levl:
                    ten, layer_ten = lay(ten, True)
                    # we need the layer representations just for the original and reconstructed,
                    # flatten, because it's a convolutional shape
                    layer_ten = layer_ten.view(len(layer_ten), -1)
                    return layer_ten
                else:
                    ten = lay(ten)
        else:
            ten = torch.cat((ten_orig, ten_predicted, ten_sampled), 0)
            for i, lay in enumerate(self.conv):
                    ten = lay(ten)

            ten = ten.view(len(ten), -1)
            ten = self.fc(ten)
            return F.sigmoid(ten)


    def __call__(self, *args, **kwargs):
        return super(Discriminator, self).__call__(*args, **kwargs)

In [4]:
disc = Discriminator()

In [ ]:
# print(summary(conv_encoder, torch.zeros((1, 3, 32, 32)).cuda(), show_input=False, show_hierarchical=False))
# print(summary(conv_decoder, torch.zeros((1, 1, 100)).cuda(), show_input=False, show_hierarchical=False))
print(summary(disc, torch.zeros((1, 1, 32, 32)).cuda(), show_input=False, show_hierarchical=False))